<a href="https://colab.research.google.com/github/heebonpark/-_-/blob/main/%EC%A0%95%EC%B9%98%EC%B4%9D%EA%B4%84%EA%B4%80%EB%A6%AC%EC%95%B1_%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C%EC%A0%9C%EC%99%B8_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

// === 환경설정 ===
const SHEET_NAME = '9월5일기준';

// 웹앱 엔트리
function doGet() {
  return HtmlService.createHtmlOutputFromFile('index')
    .setTitle('부실가입자 활동 등록')
    // 보안 강화를 위해 ALLOWALL 대신 DEFAULT를 권장합니다.
    // 필요에 따라 적절한 X-Frame-Options 모드를 설정하세요.
    .setXFrameOptionsMode(HtmlService.XFrameOptionsMode.DEFAULT); // Security Recommendation: Changed from ALLOWALL
}

/* ========= 공통 유틸 ========= */
function _norm(s){ return String(s||'').toLowerCase().replace(/\s+/g,''); }
function _findHeaderIndex(headers, candidates){
  const normMap = headers.map(h => _norm(h));
  for (let i=0;i<candidates.length;i++){
    const idx = normMap.indexOf(_norm(candidates[i]));
    if (idx !== -1) return idx;
  }
  return -1;
}
function _ensureHeader(sheet, headers, canonicalName, altNames){
  const all = [canonicalName].concat(altNames||[]);
  let idx = _findHeaderIndex(headers, all);
  if (idx !== -1) return idx;
  const lastCol = sheet.getLastColumn();
  sheet.insertColumnAfter(lastCol);
  sheet.getRange(1, lastCol + 1).setValue(canonicalName);
  headers.push(canonicalName); // Add to the headers array in memory
  return headers.length - 1; // Return the new index (0-based)
}
function formatDate_(v){
  if (!v) return '';
  try{
    if (v instanceof Date){
      return Utilities.formatDate(v, SpreadsheetApp.getActive().getSpreadsheetTimeZone(), 'yyyy-MM-dd');
    }
    const d = new Date(v);
    if (!isNaN(d.getTime())){
      return Utilities.formatDate(d, SpreadsheetApp.getActive().getSpreadsheetTimeZone(), 'yyyy-MM-dd');
    }
  }catch(e){
    Logger.log("Error formatting date: " + e);
    return '';
  }
}
function formatFee_(num){ if (typeof num!=='number'||isNaN(num)) return ''; return num.toLocaleString('ko-KR'); }

// Helper function to safely get column value
function getColumnValue(row, headers, candidates) {
  const index = _findHeaderIndex(headers, candidates);
  // Return value as is if index is -1, handled by caller or subsequent checks
  return index !== -1 ? (row[index] || '') : '';
}

function _rowToObj(headers, row){
  const o={}; headers.forEach((h,j)=>o[h]=row[j]);
  const rawFee = o['KTT월정료']; let feeNum=0;
  // Improved fee parsing
  if (typeof rawFee==='number') {
      feeNum = rawFee;
  } else if (typeof rawFee==='string'){
      const cleaned = rawFee.replace(/[^\d.-]/g,'');
      feeNum = parseFloat(cleaned)||0;
  } else {
      feeNum = 0; // Handle other types gracefully
  }

  const mapUrl = (typeof o['지도링크_URL']==='string') ? o['지도링크_URL'] : '';

  // Extract all required columns using getColumnValue for robustness
  const branch = getColumnValue(row, headers, ['지사']);
  const manager = getColumnValue(row, headers, ['지사_구역담당']);
  const name = getColumnValue(row, headers, ['상호', '고객명']);
  const contract = getColumnValue(row, headers, ['계약번호']);
  const address = getColumnValue(row, headers, ['설치주소']);
  const queryType = getColumnValue(row, headers, ['조회구분']);
  // Ensure note and visit are read correctly
  const note = getColumnValue(row, headers, ['상담내역 및 사유']);
  const visit = getColumnValue(row, headers, ['활동내역']);

  const firstStartDate = formatDate_(getColumnValue(row, headers, ['최초서비스개시일']));
  const startDateTime = formatDate_(getColumnValue(row, headers, ['서비스개시일시']));
  const endDateTime = formatDate_(getColumnValue(row, headers, ['서비스종료일시']));
  const eventStartDate = formatDate_(getColumnValue(row, headers, ['이벤트시작일']));

  const serviceType = getColumnValue(row, headers, ['L/i형']);
  const 위도 = getColumnValue(row, headers, ['위도']); // Keep original names for direct mapping
  const 경도 = getColumnValue(row, headers, ['경도']);
  const actionType = getColumnValue(row, headers, ['조치구분']);
  const actionPlan = getColumnValue(row, headers, ['조치세부계획']);
  const actionMonth = getColumnValue(row, headers, ['조치월']);
  const kpiSept = getColumnValue(row, headers, ['KPI차감 9월말']);
  const kpiOct = getColumnValue(row, headers, ['KPI차감 10월말']);
  // Ensure dispatchSecurity is read correctly - it seems it was already included
  const dispatchSecurity = getColumnValue(row, headers, ['출동보안구분', '보안구분']);
  const channel = getColumnValue(row, headers, ['실적채널']);
  // Get '체납구분' data
  const delinquency = getColumnValue(row, headers, ['체납구분']);


  // Add logging to check the extracted channel value
  Logger.log("_rowToObj: Extracted channel value: %s, Delinquency value: %s", channel, delinquency);


  return {
    branch: branch,
    manager: manager,
    name: name,
    contract: contract,
    // monthlyFee: formatFee_(feeNum), // Formatting moved to client side for display
    feeNum: feeNum, // Pass the number for calculations
    address: address,
    mapUrl: mapUrl,
    queryType: queryType,
    note: note,
    visit: visit,
    firstStartDate: firstStartDate,
    startDateTime: startDateTime,
    endDateTime: endDateTime,
    eventStartDate: eventStartDate,
    serviceType: serviceType,
    위도: 위도,
    경도: 경도,
    actionType: actionType,
    actionPlan: actionPlan,
    actionMonth: actionMonth,
    kpiSept: kpiSept,
    kpiOct: kpiOct,
    dispatchSecurity: dispatchSecurity, // Include in the returned object
    channel: channel, // Include in the returned object
    delinquency: delinquency // Include '체납구분' in the returned object
  };
}

/* ========= 전체 로드 ========= */
function getSheetData() {
  const ss = SpreadsheetApp.getActiveSpreadsheet();
  const sheet = ss.getSheetByName(SHEET_NAME);
  if (!sheet) {
    Logger.log("Sheet not found: " + SHEET_NAME);
    return []; // Return empty array if sheet not found
  }
  const values = sheet.getDataRange().getValues();
  if (values.length < 2) {
     Logger.log("Sheet is empty or has only headers: " + SHEET_NAME);
     return []; // Return empty array if no data rows
  }
  let headers = values[0];

  // Ensure '체납구분' header exists
  let initialHeaderCount = headers.length;
  _ensureHeader(sheet, headers, '체납구분', ['체납 구분', '체납']);
   if (headers.length > initialHeaderCount) {
      sheet.getRange(1, 1, 1, headers.length).setValues([headers]); // Update header row in sheet
      Logger.log("Added missing '체납구분' header.");
      // Re-read headers after updating the sheet to ensure correct indices
      headers = sheet.getRange(1,1,1,sheet.getLastColumn()).getValues()[0];
   }


  // Log headers to check for '실적채널' and '체납구분' and their positions
  Logger.log("getSheetData: Headers read: %s", headers.join(', '));


  const out = [];
  // Process data rows starting from the second row (index 1)
  for (let i=1;i<values.length;i++){
    const base = _rowToObj(headers, values[i]);
    const rowIndexInSheet = i+1; // 1-based index for the sheet row
    out.push({ ...base, rowIndex: rowIndexInSheet, originalRowIndex: rowIndexInSheet });
  }
  Logger.log("getSheetData: Returning %s rows", out.length);
  if (out.length > 0) {
    // Log the channel and delinquency value for the first few rows to verify data extraction
    Logger.log("getSheetData: First 5 rows channel and delinquency values: %s", JSON.stringify(out.slice(0, 5).map(d => ({channel: d.channel, delinquency: d.delinquency}))));
  } else {
    Logger.log("getSheetData: No data rows to return example.");
  }
  return out;
}

/* ========= 조건검색(상호, 계약번호, 설치주소 적용) ========= */
function searchData(filters){
  // Basic input validation for filters
  if (!filters || typeof filters !== 'object') {
      Logger.log("Invalid filters object provided to searchData.");
      // Optionally return empty or all data depending on desired behavior
      return getSheetData(); // Return all data if filters are invalid
  }

  const qName = _norm(filters.name || '');
  const qContract = _norm(filters.contract || '');
  const qAddress = _norm(filters.address || '');

  // Extract all filter values
  const qBranch = filters.branch || '';
  const qManager = filters.manager || '';
  const qQueryType = filters.queryType || '';
  const qFeeFilter = filters.feeFilter || '';
  const qServiceType = filters.serviceType || '';
  const qKpiSept = filters.kpiSept || '';
  const qKpiOct = filters.kpiOct || '';
  const qDispatchSecurity = filters.dispatchSecurity || '';
  const qChannel = filters.channel || '';
  const qDelinquency = filters.delinquency || ''; // Variable for '체납구분' filter

  // If no filters are provided, return all data
  if (!qName && !qContract && !qAddress && !qBranch && !qManager && !qQueryType && !qFeeFilter && !qServiceType && !qKpiSept && !qKpiOct && !qDispatchSecurity && !qChannel && !qDelinquency) {
    Logger.log("searchData: No filters provided, returning all data.");
    return getSheetData();
  }

  const ss = SpreadsheetApp.getActiveSpreadsheet();
  const sheet = ss.getSheetByName(SHEET_NAME);
  if (!sheet) {
      Logger.log("Sheet not found during search: " + SHEET_NAME);
      return []; // Return empty array if sheet not found
  }
  const values = sheet.getDataRange().getValues();
  if (values.length < 2) {
      Logger.log("Sheet is empty or has only headers during search: " + SHEET_NAME);
      return []; // Return empty array if no data rows
  }
  const headers = values[0];

  // Find indices for all columns used in filtering
  const idxName = _findHeaderIndex(headers, ['상호','고객명']);
  const idxContract = _findHeaderIndex(headers, ['계약번호']);
  const idxAddress = _findHeaderIndex(headers, ['설치주소']);
  const idxBranch = _findHeaderIndex(headers, ['지사']);
  const idxManager = _findHeaderIndex(headers, ['지사_구역담당']);
  const idxQueryType = _findHeaderIndex(headers, ['조회구분']);
  const idxFee = _findHeaderIndex(headers, ['KTT월정료']);
  const idxServiceType = _findHeaderIndex(headers, ['L/i형']);
  const idxKpiSept = _findHeaderIndex(headers, ['KPI차감 9월말']);
  const idxKpiOct = _findHeaderIndex(headers, ['KPI차감 10월말']);
  const idxDispatchSecurity = _findHeaderIndex(headers, ['출동보안구분', '보안구분']);
  const idxChannel = _findHeaderIndex(headers, ['실적채널']);
  const idxDelinquency = _findHeaderIndex(headers, ['체납구분']);


  Logger.log("searchData: Received filters: %s", JSON.stringify(filters));
  Logger.log("searchData: Filtering based on headers: %s", headers.join(', '));


  const out = [];
  for (let i=1;i<values.length;i++){
    const row = values[i];
    const rowObj = _rowToObj(headers, row); // Get row object to easily access values by key

    let passFilter = true;

    // Apply all filters server-side
    if (qName && !_norm(rowObj.name).includes(qName)) passFilter = false;
    if (passFilter && qContract && _norm(rowObj.contract) !== qContract) passFilter = false;
    if (passFilter && qAddress && !_norm(rowObj.address).includes(qAddress)) passFilter = false;
    if (passFilter && qBranch && rowObj.branch !== qBranch) passFilter = false;
    if (passFilter && qManager && rowObj.manager !== qManager) passFilter = false;
    if (passFilter && qQueryType && rowObj.queryType !== qQueryType) passFilter = false;

    if (passFilter && qFeeFilter) {
        const fee = rowObj.feeNum || 0;
        if (qFeeFilter === 'over' && fee < 100000) passFilter = false;
        if (passFilter && qFeeFilter === 'under' && fee >= 100000) passFilter = false;
    }

    const svc = (rowObj.serviceType || '').trim();
    if (passFilter && qServiceType) {
      if (qServiceType === '기타/미지정') {
        if (['L형', 'i형'].includes(svc)) passFilter = false;
      } else {
        if (svc !== qServiceType) passFilter = false;
      }
    }

    if (passFilter && qKpiSept && rowObj.kpiSept !== '대상') passFilter = false;
    if (passFilter && qKpiOct && rowObj.kpiOct !== '대상') passFilter = false;

    const dispatch = (rowObj.dispatchSecurity || '').trim();
     if (passFilter && qDispatchSecurity) {
        if (qDispatchSecurity === '기타/미지정') {
             if (['출동보안', '영상보안'].includes(dispatch)) passFilter = false;
        } else {
             if (dispatch !== qDispatchSecurity) passFilter = false;
        }
     }

    const chan = (rowObj.channel || '').trim();
    if (passFilter && qChannel) {
        if (qChannel === '기타/미지정') {
             if (['SP', 'SC', 'AM'].includes(chan)) passFilter = false;
        } else {
             if (chan !== qChannel) passFilter = false;
        }
    }

    const del = (rowObj.delinquency || '').trim();
    if (passFilter && qDelinquency) {
        if (qDelinquency === '기타/미지정') {
            if (['일반', '체납'].includes(del)) passFilter = false;
        } else {
            if (del !== qDelinquency) passFilter = false;
        }
    }


    if (passFilter){
      const rowIndexInSheet = i+1; // 1-based index
      out.push({ ...rowObj, rowIndex: rowIndexInSheet, originalRowIndex: rowIndexInSheet });
    }
  }
  Logger.log("searchData: Returning %s filtered rows", out.length);
  return out;
}

/* ========= 저장 ========= */
function saveRowData(data) { // Renamed function to match client-side call
  const ss = SpreadsheetApp.getActiveSpreadsheet();
  const sheet = ss.getSheetByName(SHEET_NAME);

  if (!sheet) {
    Logger.log("Error: Sheet '" + SHEET_NAME + "' not found during save.");
    throw new Error("저장 실패: 시트 '" + SHEET_NAME + "'를 찾을 수 없습니다.");
  }

  const rowIndexInSheet = data.rowIndex;
  // Get the data fields from the input object sent from the client
  const actionType = data.actionType; // Value from '활동 유형' dropdown
  const actionPlan = data.actionPlan; // Value from '활동 계획' textarea
  const actionMonth = data.actionMonth; // Value from '활동 월' dropdown

  let rowIdx;
  try {
    rowIdx = parseInt(rowIndexInSheet, 10);
    if (isNaN(rowIdx) || rowIdx < 2 || rowIdx > sheet.getLastRow()) {
      Logger.log("Error: Invalid row number provided for saving: " + rowIndexInSheet + " (Last row: " + sheet.getLastRow() + ")");
      throw new Error("저장 실패: 유효하지 않은 행 번호입니다. 목록을 새로고침 해주세요. (행 번호: " + rowIndexInSheet + ")");
    }
  } catch (e) {
    Logger.log("Error processing row index " + rowIndexInSheet + " during save: " + e);
    throw new Error("저장 실패: 행 번호를 처리하는 중 오류가 발생했습니다. (" + (e.message || e) + ")");
  }

  // --- Validation ---
  // Validate actionPlan as it maps to '상담내역 및 사유' (Column Q)
  if (!actionPlan || typeof actionPlan !== 'string' || actionPlan.trim() === '') {
      Logger.log("Validation Error: '활동 계획' (상담내역 및 사유) is empty or invalid for row " + rowIdx);
      throw new Error("저장 실패: '활동 계획' 필드를 입력해주세요.");
  }
  // Optional validation for actionType and actionMonth if needed

  // --- Header Handling and Data Writing ---
  // Get current headers to find indices for headers we still need to look up.
  // We will use fixed indices for '활동내역' (P) and '상담내역 및 사유' (Q).
  let headers = sheet.getRange(1,1,1,sheet.getLastColumn()).getValues()[0];

  // Define fixed column indices for '활동내역' (P) and '상담내역 및 사유' (Q)
  const COL_IDX_ACTIVITY = 15; // P column is the 16th column (index 15)
  const COL_IDX_NOTE = 16;     // Q column is the 17th column (index 16)

  // Find or ensure headers for the other fields being saved ('조치구분', '조치세부계획', '조치월', '방문일')
  // Use _ensureHeader for these as they are not explicitly fixed to P/Q.
  // Re-read headers after ensuring columns in case new ones were added.
  let initialHeaderCount = headers.length;
  let idxActionType = _ensureHeader(sheet, headers, '조치구분', ['조치 유형']);
  let idxActionPlan = _ensureHeader(sheet, headers, '조치세부계획', ['세부 계획']);
  let idxActionMonth = _ensureHeader(sheet, headers, '조치월', ['활동 월']);
  let idxVisitDate = _ensureHeader(sheet, headers, '방문일', ['방문일자','저장일','기록일','방문 일자']);

  // If columns were added, re-read headers to get correct indices
  if (headers.length > initialHeaderCount) {
      sheet.getRange(1, 1, 1, headers.length).setValues([headers]); // Update header row in sheet
      Logger.log("Added missing headers during save: " + headers.slice(initialHeaderCount).join(', '));
      // Re-read headers after updating the sheet
      headers = sheet.getRange(1,1,1,sheet.getLastColumn()).getValues()[0];
      // Re-find indices after updating headers
      idxActionType = _findHeaderIndex(headers, ['조치구분', '조치 유형']);
      idxActionPlan = _findHeaderIndex(headers, ['조치세부계획', '세부 계획']);
      idxActionMonth = _findHeaderIndex(headers, ['조치월', '활동 월']);
      idxVisitDate = _findHeaderIndex(headers, ['방문일', '방문일자','저장일','기록일','방문 일자']);
   }

   // Add a check to ensure indices for looked-up headers are found
   if (idxActionType === -1 || idxActionPlan === -1 || idxActionMonth === -1 || idxVisitDate === -1) {
       Logger.log("Error: Could not find required headers for dynamic columns. idxActionType: %s, idxActionPlan: %s, idxActionMonth: %s, idxVisitDate: %s", idxActionType, idxActionPlan, idxActionMonth, idxVisitDate);
       throw new Error("저장 실패: 필수 헤더를 찾을 수 없습니다. 시트 구조를 확인하거나 관리자에게 문의하세요.");
   }


  const lock = LockService.getDocumentLock();
  lock.waitLock(20 * 1000); // Increased wait time slightly

  try {
    // Check if the row still exists before writing - basic check
    // Get the range for the row we're about to write to. This will throw an error if the row doesn't exist.
    sheet.getRange(rowIdx, 1).activate(); // Attempt to activate a cell in the row to check existence

    Logger.log("saveRowData: Saving data for row %s. Writing to fixed P/Q and dynamic columns.", rowIdx);
    Logger.log("Data: actionType='%s', actionPlan='%s', actionMonth='%s'", actionType, actionPlan, actionMonth);
    Logger.log("Dynamic Column Indices: actionType=%s, actionPlan=%s, actionMonth=%s, visitDate=%s", idxActionType, idxActionPlan, idxActionMonth, idxVisitDate);
    Logger.log("Fixed Column Indices: Activity (P)=%s, Note (Q)=%s", COL_IDX_ACTIVITY, COL_IDX_NOTE);


    // Write to fixed columns P ('활동내역') and Q ('상담내역 및 사유')
    sheet.getRange(rowIdx, COL_IDX_ACTIVITY + 1).setValue(actionType ? actionType.trim() : ''); // Save Action Type to P
    sheet.getRange(rowIdx, COL_IDX_NOTE + 1).setValue(actionPlan ? actionPlan.trim() : '');       // Save Action Plan to Q

    // Write to dynamically found columns ('조치구분', '조치세부계획', '조치월')
    sheet.getRange(rowIdx, idxActionType + 1).setValue(actionType ? actionType.trim() : ''); // Save Action Type to '조치구분'
    sheet.getRange(rowIdx, idxActionPlan + 1).setValue(actionPlan ? actionPlan.trim() : '');       // Save Action Plan to '조치세부계획'
    sheet.getRange(rowIdx, idxActionMonth + 1).setValue(actionMonth ? actionMonth.trim() : ''); // Save Action Month to '조치월'


    sheet.getRange(rowIdx, idxVisitDate + 1).setValue(new Date()); // Update Visit Date on save


    SpreadsheetApp.flush(); // Ensure changes are written immediately

    const ts = Utilities.formatDate(new Date(), ss.getSpreadsheetTimeZone(), 'yyyy-MM-dd HH:mm:ss');
    Logger.log("Successfully saved data for row " + rowIdx + " at " + ts);
    return { 'ok': true, 'ts': ts };

  } catch (e) {
    Logger.log("Error saving data to row " + rowIdx + ": " + e);
    // Check for common errors like deleted rows or invalid ranges
    if (e.message && (e.message.includes("Range not found") || e.message.includes("row out of range"))) {
         throw new Error("저장 실패: 선택한 데이터 행(" + rowIdx + "행)이 시트에서 삭제된 것 같습니다. 목록을 새로고침 해주세요.");
    }
    // Re-throw other errors with a general message and details
    throw new Error("저장 실패 (행 " + rowIdx + "): 데이터 쓰기 중 오류가 발생했습니다. 잠시 후 다시 시도하거나 관리자에게 문의하세요. 오류 상세: " + (e.message || e));
  } finally {
    if (lock.hasLock()) { // Check if the lock was successfully acquired before releasing
        lock.releaseLock();
    }
  }
}

In [ ]:
<!DOCTYPE html>
<html>
<head>
  <base target="_top">
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <style>
    :root{
      --bg1:#eef4f9; --bg2:#f7fbff;
      --brand:#0b3b5a; --section:#0d5f85; --accent:#ffe44d;
      --card:#ffffff; --muted:#6b7280;
      --barA:#00bcd4; --barB:#f06292; --line:#3b82f6;
    }
    body{ font-family:'Segoe UI',system-ui,-apple-system,'Apple SD Gothic Neo','Noto Sans KR',sans-serif;
      margin:0; display:flex; height:100vh; flex-direction:row; background:linear-gradient(180deg,var(--bg1),var(--bg2));
    }
    .sidebar{ background:var(--brand); color:#fff; padding:1rem; width:320px; display:flex; flex-direction:column; gap:1rem; overflow-y:auto; box-shadow:2px 0 6px rgba(0,0,0,.15); flex-shrink:0; }
    .section{ border:1px solid rgba(255,255,255,.2); border-radius:12px; padding:1rem; background-color:var(--section); }
    .section-title{ font-size:1.02rem; font-weight:800; margin-bottom:.8rem; color:#fff; letter-spacing:.2px; }

    .button-group{ display:grid; grid-template-columns:repeat(2,minmax(0,1fr)); gap:.5rem; }
    .btn{ padding:.55rem .65rem; border:none; border-radius:10px; font-size:.94rem; cursor:pointer; width:100%;
      text-align:center; transition:filter .15s ease, transform .02s ease; white-space:normal; line-height:1.3; min-height:42px; background:#0ea5e9; color:#fff;
    }
    .btn:hover{ filter:brightness(1.06); transform:translateY(-1px); }
    .btn-branch{ background:#117aa6; } .btn-manager{ background:#33aa77; } .btn-query{ background:#0099aa; }
    .btn-visualization{ background:#e0aaff; color:#311b4f; font-weight:800; }
    .btn-kpi{ background:#ffc107; color:#333; font-weight:700; }
    .btn-dispatch{ background:#8c52ff; color:#fff; font-weight:700; }
    /* New style for Channel buttons */
    .btn-channel{ background:#4caf50; color:#fff; font-weight:700; }

    .btn.active{ outline:3px solid var(--accent); box-shadow:0 0 8px var(--accent); }

    .main{ flex:1; padding:1rem 1.25rem; overflow-y:auto; display:flex; flex-direction:column; position:relative; }
    h2{ margin:.2rem 0 1rem 0 }

    /* 검색 바 */
    .search-bar{ background:var(--card); border:1px solid #e5e7eb; border-radius:12px; padding:.8rem; margin-bottom:.8rem;
      display:grid; grid-template-columns:1fr 1fr 1fr auto auto; gap:.5rem; align-items:center;}
    .search-input{ width:100%; padding:.55rem .65rem; border:1px solid #d1d5db; border-radius:8px; font-size:.95rem; }
    .chip-wrap{ grid-column:1 / -1; display:flex; flex-wrap:wrap; gap:.4rem; }
    .chip{ display:inline-flex; align-items:center; gap:.35rem; background:#eef2ff; color:#334155; border:1px solid #c7d2fe; padding:.2rem .55rem; border-radius:999px; font-size:.85rem; }

    .info-banner{ background:#fffbea; border:1px solid #ffe58f; color:#8b6d00; border-radius:10px; padding:.7rem 1rem; margin:0 0 .8rem 0; font-size:.95rem; }

    /* 가이드(접기/펴기) */
    .guide{ background:#fff; border:1px solid #e5e7eb; border-radius:12px; padding:.9rem 1rem; margin-bottom:1rem; }
    .guide-toggle{ display:flex; justify-content:space-between; align-items:center; gap:.75rem; cursor:pointer; padding:.4rem .5rem; border-radius:8px; transition:background .15s ease,color .15s ease; background:#f1f5f9; color:#0f172a; }
    .guide.open .guide-toggle{ background:#0ea5e9; color:#fff; }
    .guide-title{ font-weight:800; }
    .guide-body{ margin-top:.7rem; color:#374151; font-size:.92rem; line-height:1.5; display:none; }
    .guide.open .guide-body{ display:block; }

    /* 자료 바로가기 */
    .resource-bar{ display:flex; gap:.8rem; flex-wrap:wrap; margin:.6rem 0 .9rem 0; }
    .resource-button{ display:inline-block; background:#0ea5e9; color:#fff; text-decoration:none; padding:.6rem .85rem; border-radius:10px; font-weight:800; border:1px solid rgba(255,255,255,.15); box-shadow:0 4px 10px rgba(14,165,233,.25); cursor:pointer; position:relative; }
    .resource-button.secondary{ background:#22c55e; } .resource-button.warning{ background:#f59e0b; }
    .resource-button .dropdown-content { display: none; position: absolute; background-color: #f9f9f9; min-width: 200px; box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2); z-index: 1; border-radius: 8px; overflow: hidden; }
    .resource-button:hover .dropdown-content { display: block; }
    .resource-button .dropdown-content a { color: black; padding: 10px 14px; text-decoration: none; display: block; font-weight: normal; font-size: 0.9rem; }
    .resource-button .dropdown-content a:hover { background-color: #f1f1f1; }


    /* 차트: 동일 비율 */
    .chart-container{ display:flex; flex-direction:row; gap:1.2rem; margin-bottom:1.2rem; flex-wrap:wrap; justify-content:center; }
    .chart-item{ flex:1; min-width:340px; max-width:50%; background:var(--card); border-radius:12px; padding:1rem; box-shadow:0 2px 8px rgba(0,0,0,.08); box-sizing:border-box; }
    .chart-container.single-chart .chart-item{ max-width:100%; }
    .chart-viewport{ width:100%; height:auto; }
    .chart-viewport.bar{ aspect-ratio: 1 / 1; }   /* 데스크톱 */
    .chart-viewport.radar{ aspect-ratio: 1 / 1; }
    canvas{ width:100% !important; height:100% !important; display:block; }

    .card{ background:var(--card); padding:1.1rem 1.2rem; margin-bottom:1rem; border-radius:12px; box-shadow:0 2px 8px rgba(0,0,0,.08); border-left:6px solid #007bff; }
    .card-title{ display:flex; align-items:center; gap:.5rem; margin:0 0 .5rem 0; font-size:1.03rem; font-weight:700; line-height:1.3; flex-wrap:wrap; }
    .badge-note{ display:inline-flex; align-items:center; gap:.35rem; font-size:.82rem; background:#eef2ff; color:#334155; border:1px solid #c7d2fe; padding:.15rem .5rem; border-radius:999px; font-size:.85rem; }
    .contract{ padding:.8rem .95rem; background:#eaf3ff; border-left:5px solid #007bff; margin-top:.45rem; border-radius:8px; }
    .contract p{ margin:.42rem 0 }
    label{ display:block; margin-top:.75rem; margin-bottom:.45rem; font-weight:700; color:#333; }
    select, textarea{ width:100%; padding:.75rem; font-size:1rem; border-radius:8px; box-sizing:border-box; border:1px solid #d1d5db; }
    textarea{ resize:vertical; }
    .save-row{ display:flex; gap:.6rem; align-items:center; margin-top:.6rem; flex-wrap:wrap; }
    .last-save{ font-size:.86rem; color:var(--muted) }
    .save-btn{ background:#d9534f; color:#fff; padding:.55rem 1rem; border:none; border-radius:10px; cursor:pointer; font-size:1rem; }
    .save-btn:disabled{ opacity:.6; cursor:not-allowed }

    .overlay{ position:fixed; inset:0; background:rgba(255,255,255,.92); display:flex; align-items:center; justify-content:center; z-index:2000; font-size:1.05rem; color:#333; backdrop-filter:blur(1px); text-align:center; padding:0 1rem; }
    .spinner{ width:20px; height:20px; border:3px solid #ddd; border-top-color:#007bff; border-radius:50%; margin-right:10px; animation: spin .8s linear infinite; display:inline-block; vertical-align:middle; }
    @keyframes spin{ to{ transform:rotate(360deg); } }
    #toast{ visibility:hidden; min-width:280px; background:#333; color:#fff; text-align:center; border-radius:10px; padding:12px 14px; position:fixed; bottom:30px; left:50%; transform:translateX(-50%); z-index:2200; opacity:0; transition:opacity .4s, visibility .4s; }
    #toast.show{ visibility:visible; opacity:1; }

    @media (max-width:768px){
      body{ flex-direction:column; }
      .sidebar{ width:100%; flex-direction:row; flex-wrap:wrap; padding:.8rem; gap:.8rem; }
      .section{ width:100%; padding:.8rem; }
      .button-group{ grid-template-columns:repeat(3,minmax(0,1fr)); }
      .btn{ font-size:.92rem; padding:.55rem; }
      .main{ padding:.8rem; }
      .search-bar{ grid-template-columns:1fr 1fr; grid-auto-rows:auto; }
      .chart-container{ flex-direction:column; gap:1rem; }
      .chart-item{ min-width:auto; max-width:100%; }
      .chart-viewport.bar{ aspect-ratio: 4 / 5; }   /* 모바일 */
      .chart-viewport.radar{ aspect-ratio: 4 / 5; }
    }
  </style>
</head>
<body>
  <!-- 로딩/저장 오버레이 -->
  <div id="loadingOverlay" class="overlay" style="display:none"><div><div><span class="spinner"></span><span id="overlayMsg">잠시만 기다려주세요… 데이터 로딩 중입니다.</span></div><div style="margin-top:.75rem;"><button class="btn" style="background:#007bff" onclick="retryLoad()">다시 시도</button></div></div></div>
  <div id="savingOverlay" class="overlay" style="display:none"><div><span class="spinner"></span><span>저장중입니다. 잠시만 기다려주세요…</span></div></div>

  <!-- 사이드바 -->
  <div class="sidebar">
    <div class="section"><div class="section-title">📊 시각화 선택</div><div id="visualizationBox" class="button-group"></div></div>
    <div class="section"><div class="section-title">실적채널</div><div id="channelBox" class="button-group"></div></div> <!-- New Channel section -->
    <div class="section"><div class="section-title">KPI 차감</div><div id="kpiBox" class="button-group"></div></div>
    <div class="section"><div class="section-title">📦 L/i형</div><div id="serviceTypeBox" class="button-group"></div></div>
    <div class="section"><div class="section-title">📦 출동보안구분</div><div id="dispatchSecurityBox" class="button-group"></div></div>
    <div class="section"><div class="section-title">📂 조회구분</div><div id="queryBox" class="button-group"></div></div>
    <div class="section"><div class="section-title">💰 월정료 조건</div><div id="feeBox" class="button-group"></div></div>
    <div class="section"><div class="section-title">📍 지사</div><div id="branchBox" class="button-group"></div></div>
    <div class="section"><div class="section-title">👤 구역담당</div><div id="managerBox" class="button-group"></div></div>

  </div>

  <!-- 메인 -->
  <div class="main">
    <h2>📋 일시정지/부실가입자 활동 관리</h2>

    <!-- 검색 바 -->
    <div class="search-bar">
      <input type="text" id="qName" class="search-input" placeholder="상호 일부 검색">
      <button id="btnSearch" class="btn" style="background:#28a745;color:#fff;">검색</button>
      <button id="btnReset" class="btn" style="background:#6c757d;color:#fff;">초기화</button>
      <div id="chipWrap" class="chip-wrap"></div>
      <!-- Add hidden inputs for contract and manager search if needed in the future -->
      <input type="hidden" id="qContract">
      <input type="hidden" id="qManager">
    </div>


    <div class="info-banner">ℹ️ <b>실적채널/지사/구역담당/조회구분/L·i형/출동보안구분/월정료/KPI 차감</b> 버튼으로 필터하세요. 상단에서 <b>건수/월정료 시각화</b> 전환도 가능합니다. (검색은 <b>상호 일부</b>만 적용)</div> <!-- Updated info banner -->

    <!-- 접기/펴기 안내 -->
    <div id="guide" class="guide">
      <div class="guide-toggle" onclick="toggleGuide()">
        <div class="guide-title">📌 정지시설 범위/기준 · KPI 변경 · 우선순위 (접기/펴기)</div>
        <div id="guideIcon">▼</div>
      </div>
      <div class="guide-body">
        <ul>
          <li><b>정지시설의 범위와 기준</b>
            <ul>
              <li>리텐션팀 확정 정지마감시설 리스트 기준 활용</li>
              <li>대상: 「정지사유」=<b>KTT일반정지</b>, <b>KTT 직권정지</b> / <b>KT시설 제외</b>(i형 포함)</li>
              <li>채널: 「담당채널」=<b>SP/SC/AM</b></li>
            </ul>
          </li>
          <li><b>KPI 평가방식 변경</b>
            <ul>
              <li>기존: 당월 순증월정료 − 당월 마감 <u>장기정지 진입</u></li>
              <li>변경: 평가월 <u>누적 순증월정료</u> − 해당월 마감 <u>장기정지(90일↑)</u></li>
            </ul>
          </li>
          <li><b>정지가입자 관리활동 우선순위</b>
            <ul>
              <li>유형: <b>고ARPU</b> &gt; <b>장기정지</b> &gt; <b>체납직권정지</b> &gt; 일반정지</li>
              <li>활동: <b>유지전환</b> 우선 &gt; 미사용 의사 시 <b>해지정리</b></li>
              <li><b>KT i형</b>은 지역본부/지사 정지관리 <b>대상 제외</b></li>
            </ul>
          </li>
        </ul>
        <div style="margin-top:.4rem;color:#0f172a"><b>▶ 지역본부/지사</b>는 <b>L형</b> 가입자 정지관리에 집중해 주세요.</div>
      </div>
    </div>

    <!-- 자료 바로가기 -->
    <div class="resource-bar" aria-label="관련 자료 바로가기">
      <div class="resource-button warning">📊 지표/현황 바로가기
        <div class="dropdown-content">
          <a target="_blank" rel="noopener" href="https://script.google.com/macros/s/AKfycbwkWSXQIQEBkE0DGRs6YC9ObDyyzdBHbGH48zUC7PCKHkWzEcLqBuvVH2W6WSmyuJNCAw/exec">월별 정지/부실율 추이 현황</a>
          <a target="_blank" rel="noopener" href="https://script.google.com/macros/s/AKfycbxCA-qrDUz5eEr8g-2QxRi8NnrVX9Rok2xzmL0ll5KUt0TGeVrUfFNjKbESzsW-M1ZxPQ/exec">기관별 정지/부실 건수/월정료 현황</a>
        </div>
      </div>
      <div class="resource-button secondary">📎 관련자료 다운로드
        <div class="dropdown-content">
          <a target="_blank" rel="noopener" href="https://drive.google.com/file/d/1mT_tpZ9rZgffa8nGtCCb3n2k-bheyHn6/view?usp=sharing">KPI_장기정지 설명자료(8.14)</a>
          <a target="_blank" rel="noopener" href="https://drive.google.com/file/d/1qHDUPq57LRrjkvwUYb-UcAgymChgQOc0/view?usp=drive_link">정지관리·조치활동 G/L</a>
          <a target="_blank" rel="mdocs?id=1gRltray9BuEbKN_Lx7znYSz0OI-dcTQ6&export=html" rel="noopener" href="https://drive.google.com/file/d/1gRltray9BuEbKN_Lx7znYSz0OI-dcTQ6/view?usp=drive_link">KPI 순증월정료 차감기준(안)</a>
        </div>
      </div>
    </div>

    <!-- 차트 -->
    <div class="chart-container" id="chartContainer">
      <div id="barChartContainer" class="chart-item">
        <div class="chart-viewport bar"><canvas id="barChart"></canvas></div>
      </div>
      <div id="radarChartContainer" class="chart-item">
        <div class="chart-viewport radar"><canvas id="radarChart"></canvas></div>
      </div>
      <!-- New containers for Manager Charts -->
      <div id="managerBarChartContainer" class="chart-item" style="display:none;">
        <div class="chart-viewport bar"><canvas id="managerBarChart"></canvas></div>
      </div>
      <div id="managerRadarChartContainer" class="chart-item" style="display:none;">
        <div class="chart-viewport radar"><canvas id="managerRadarChart"></canvas></div>
      </div>
    </div>

    <div id="resultBox"></div>
  </div>

  <div id="toast">✅ 저장 완료</div>

  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/chartjs-plugin-datalabels@2"></script>

  <script>
    // ===== 전역 =====
    let allData=[], filtered=[];
    let currentBranch='', currentManager='', currentQueryType='', feeFilter='';
    let currentServiceType='';
    let currentKpiSeptFilter='', currentKpiOctFilter='';
    let currentDispatchSecurityFilter='';
    let currentChannelFilter=''; // New variable for Channel filter
    let currentChartType='count';
    let barChartInstance=null, radarChartInstance=null;
    let managerBarChartInstance=null, managerRadarChartInstance=null; // New chart instances for managers
    const branchOrder=['중앙','강북','서대문','고양','의정부','남양주','강릉','원주'];
    const fmtK=(n)=> Math.round((n||0)/1000).toLocaleString()+'천원';
    const fmtInt=(n)=> (n||0).toLocaleString();
    const debounce=(fn,wait=200)=>{ let t; return (...a)=>{ clearTimeout(t); t=setTimeout(()=>fn(...a),wait); }; }

    // ===== 로딩 =====
    safeLoad();
    function safeLoad(){
      showOverlay('잠시만 기다려주세요… 데이터 로딩 중입니다.');
      try{
        if (!google || !google.script || !google.script.run) throw new Error('google.script.run 사용 불가');
        google.script.run.withSuccessHandler(init).withFailureHandler(handleError).getSheetData();
      }catch(err){ handleError(err); }
    }
    function retryLoad(){ safeLoad(); }
    function showOverlay(msg){ const ov=document.getElementById('loadingOverlay'); const el=document.getElementById('overlayMsg'); if(el) el.textContent=msg||'로딩 중…'; if(ov) ov.style.display='flex'; }
    function hideOverlay(){ const ov=document.getElementById('loadingOverlay'); if(ov) ov.style.display='none'; }
    function handleError(err){ console.error('[ERROR]', err); showOverlay('데이터 불러오기에 실패했습니다. 다시 시도해 주세요.'); showToast('⚠️ 데이터 불러오기 실패'); }

    // ===== 초기화 =====
    function init(data){
      try{
        console.log('init: Received data:', data);
        console.log('init: Data structure example:', data && data.length > 0 ? data[0] : 'No data');

        allData=(data||[]).map((d,idx)=>({...d,
          address:(d?.address ?? '').toString(),
          mapUrl:d?.mapUrl ?? '',
          feeNum:Number(d?.feeNum ?? 0) || 0,
          rowIndex:Number(d?.rowIndex ?? d?.originalRowIndex ?? idx+2),
          originalRowIndex:Number(d?.originalRowIndex ?? d?.rowIndex ?? idx+2),
          kpiSept: (d?.kpiSept ?? '').toString().trim(),
          kpiOct: (d?.kpiOct ?? '').toString().trim(),
          dispatchSecurity: (d?.dispatchSecurity ?? '').toString().trim(),
          channel: (d?.channel ?? '').toString().trim() // Ensure channel data is string and trimmed
        }));
        console.log('init: Processed allData:', allData);
        renderAllButtons(); applyFilter(); setChartType(currentChartType);
        if (!hasAnyFilter() && filtered.length===0){
          document.getElementById("resultBox").innerHTML='<p>👉 필터를 선택해 목록을 확인하세요.</p>';
        } else { renderResults(); }
        window.addEventListener('resize', debounce(renderCharts, 150));
      }catch(e){ handleError(e); return; } finally{ hideOverlay(); }

      // 검색(상호 일부만)
      document.getElementById('btnSearch').onclick=doSearch;
      document.getElementById('btnReset').onclick=()=>{ ['qContract','qName','qManager'].forEach(id=>document.getElementById(id).value=''); applyChips({}); doSearch(true); };
      ['qContract','qName','qManager'].forEach(id=> document.getElementById(id).addEventListener('keydown',(e)=>{ if(e.key==='Enter') doSearch(); }));
    }

    // ===== 가이드 토글 =====
    function toggleGuide(){
      const wrap=document.getElementById('guide');
      const icon=document.getElementById('guideIcon');
      wrap.classList.toggle('open');
      icon.textContent = wrap.classList.contains('open') ? '▲' : '▼';
    }

    // ===== 검색(상호 일부만 적용) =====
    function applyChips(f){ const w=document.getElementById('chipWrap'); w.innerHTML=''; if(f.name) w.innerHTML+=`<span class="chip">상호(포함): ${f.name}</span>`; }
    function doSearch(isReset){
      const name = isReset ? '' : (document.getElementById('qName').value.trim());
      if (!isReset){
        const hasOthers = (document.getElementById('qContract').value.trim() || document.getElementById('qManager').value.trim());
        if (hasOthers) showToast('현재 검색은 "상호 일부"만 적용됩니다.');
      }
      const filters = { name };
      applyChips(filters);

      const runner = name ? 'searchData' : 'getSheetData';
      showOverlay('조회 중…');
      console.log('doSearch: Calling Apps Script function:', runner, 'with filters:', filters);

      google.script.run
        .withSuccessHandler((data)=>{
          hideOverlay();
          console.log('doSearch: Received data on success:', data);
          allData=(data||[]).map((d,idx)=>({...d,
            feeNum:Number(d?.feeNum||0)||0,
            rowIndex:Number(d?.rowIndex ?? d?.originalRowIndex ?? idx+2),
            originalRowIndex:Number(d?.originalRowIndex ?? d?.rowIndex ?? idx+2),
            kpiSept: (d?.kpiSept ?? '').toString().trim(),
            kpiOct: (d?.kpiOct ?? '').toString().trim(),
            dispatchSecurity: (d?.dispatchSecurity ?? '').toString().trim(),
            channel: (d?.channel ?? '').toString().trim() // Ensure channel data is string and trimmed
          }));
          console.log('init: Processed allData:', allData);
          renderAllButtons(); applyFilter(); setChartType(currentChartType);
          showToast(`조회 완료 (${allData.length.toLocaleString()}건)`);
        })
        .withFailureHandler((e)=>{
          hideOverlay();
          console.error('doSearch: Apps Script call failed:', e);
          showToast('조회 실패: '+(e?.message||e));
        })
        [runner](filters);
    }

    // ===== 버튼 =====
    function renderAllButtons(){ renderVizButtons(); renderChannelButtons(); renderKpiButtons(); renderQueryTypeButtons(); renderServiceTypeButtons(); renderFeeFilterButtons(); renderDispatchSecurityButtons(); renderBranches(); renderManagers(); } // Added renderChannelButtons
    function hasAnyFilter(){ return !!(currentBranch||currentManager||currentQueryType||feeFilter||currentServiceType||currentKpiSeptFilter||currentKpiOctFilter||currentDispatchSecurityFilter||currentChannelFilter); } // Added Channel filter
    function setFilterFromButton(btn){
      const kind=btn.dataset.kind, value=btn.dataset.value|| '';
      if (kind==='branch'){
        currentBranch=value;
        currentManager=''; // Reset manager when branch changes
        renderManagers();
      } else if (kind==='manager'){
        currentManager=value;
      } else if (kind==='queryType'){
        currentQueryType=value;
      } else if (kind==='serviceType'){
        currentServiceType=value;
      } else if (kind==='fee'){
        feeFilter=value;
      } else if (kind==='viz'){
        currentChartType=value;
      } else if (kind==='kpiSept'){
        currentKpiSeptFilter=value;
        currentKpiOctFilter=''; renderKpiButtons(); // Clear the other KPI filter
      } else if (kind==='kpiOct'){
        currentKpiOctFilter=value;
        currentKpiSeptFilter=''; renderKpiButtons(); // Clear the other KPI filter
      } else if (kind==='dispatchSecurity'){
        currentDispatchSecurityFilter=value;
      } else if (kind==='channel'){
        currentChannelFilter=value;
      }

      highlightActive(kind, value); applyFilter(); renderCharts();
    }
    function highlightActive(kind,value){ document.querySelectorAll(`[data-kind="${kind}"]`).forEach(b=> b.classList.toggle('active',(b.dataset.value||'')===(value||''))); }
    function makeBtn(cls,label,dataset){ const b=document.createElement('button'); b.className=`btn ${cls}`; b.textContent=label; b.dataset.kind=dataset.kind; b.dataset.value=dataset.value; b.onclick=()=>setFilterFromButton(b); return b; }
    function totalCountFee(list){ let c=0,f=0; list.forEach(d=>{ if(d.contract){ c++; f+=d.feeNum||0; } }); return {c,f}; }
    function renderVizButtons(){ const box=document.getElementById('visualizationBox'); box.innerHTML=''; const {c,f}=totalCountFee(allData);
      box.appendChild(makeBtn('btn-visualization', `건수 시각화 (총 ${fmtInt(c)}건)`, {kind:'viz',value:'count'}));
      box.appendChild(makeBtn('btn-visualization', `월정료 시각화 (총 ${fmtK(f)})`, {kind:'viz',value:'fee'}));
      highlightActive('viz', currentChartType);
    }
    // Function to render Channel buttons
    function renderChannelButtons(){
        const box=document.getElementById('channelBox'); box.innerHTML='';
        const types = ['SP','SC','AM']; // Define possible values for Channel
        const counts = {};
        const fees = {};
        let totalCount = 0;
        let totalFee = 0;

        allData.filter(d => d.contract).forEach(d => {
            totalCount++;
            totalFee += d.feeNum || 0;
            const type = (d.channel || '').trim();
            if (types.includes(type)) {
                counts[type] = (counts[type] || 0) + 1;
                fees[type] = (fees[type] || 0) + (d.feeNum || 0);
            } else {
                 // Count items with other or empty channel values
                const otherKey = '기타/미지정';
                counts[otherKey] = (counts[otherKey] || 0) + 1;
                fees[otherKey] = (fees[otherKey] || 0) + (d.feeNum || 0);
            }
        });

        box.appendChild(makeBtn('btn-channel', `전체 (${fmtInt(totalCount)}건 / ${fmtK(totalFee)})`, {kind:'channel', value:''}));
        types.forEach(type => {
            box.appendChild(makeBtn('btn-channel', `${type} (${fmtInt(counts[type] || 0)}건 / ${fmtK(fees[type] || 0)})`, {kind:'channel', value:type}));
        });
         // Add button for Other/Undefined channels if any
        const otherKey = '기타/미지정';
        if(counts[otherKey] > 0) {
             box.appendChild(makeBtn('btn-channel', `${otherKey} (${fmtInt(counts[otherKey] || 0)}건 / ${fmtK(fees[otherKey] || 0)})`, {kind:'channel', value:otherKey}));
        }

        highlightActive('channel', currentChannelFilter);
    }

    // Function to render KPI buttons
    function renderKpiButtons(){
      const box=document.getElementById('kpiBox'); box.innerHTML='';
      let septCount=0, octCount=0, totalCount=0;
      let septFee=0, octFee=0, totalFee=0; // Added fee variables
      allData.filter(d=>d.contract).forEach(d=>{
        totalCount++;
        totalFee += d.feeNum || 0; // Calculate total fee
        if(d.kpiSept==='대상'){ septCount++; septFee += d.feeNum || 0; } // Calculate Sept fee
        if(d.kpiOct==='대상'){ octCount++; octFee += d.feeNum || 0; } // Calculate Oct fee
      });


      box.appendChild(makeBtn('btn-kpi', `KPI차감 9월말 (${fmtInt(septCount)}건 / ${fmtK(septFee)})`, {kind:'kpiSept', value:'대상'})); // Use kpiSept kind and include Sept fee
      box.appendChild(makeBtn('btn-kpi', `KPI차감 10월말 (${fmtInt(octCount)}건 / ${fmtK(octFee)})`, {kind:'kpiOct', value:'대상'})); // Use kpiOct kind and include Oct fee


      // Highlight active KPI button
      if(currentKpiSeptFilter === '대상') {
        highlightActive('kpiSept', '대상');
      } else if (currentKpiOctFilter === '대상') {
        highlightActive('kpiOct', '대상');
      } else {
        highlightActive('kpiSept', ''); // Highlight "전체" if no KPI filter is active
      }
    }
    // Function to render Dispatch Security buttons
    function renderDispatchSecurityButtons(){
        const box=document.getElementById('dispatchSecurityBox'); box.innerHTML='';
        const types = ['출동보안','영상보안'];
        const counts = {};
        const fees = {};
        let totalCount = 0;
        let totalFee = 0;

        allData.filter(d => d.contract).forEach(d => {
            totalCount++;
            totalFee += d.feeNum || 0;
            const type = (d.dispatchSecurity || '').trim();
            if (types.includes(type)) {
                counts[type] = (counts[type] || 0) + 1;
                fees[type] = (fees[type] || 0) + (d.feeNum || 0);
            } else if (type === '') {
                const otherKey = '기타/미지정';
                counts[otherKey] = (counts[otherKey] || 0) + 1;
                fees[otherKey] = (fees[otherKey] || 0) + (d.feeNum || 0);
            }
        });

        box.appendChild(makeBtn('btn-dispatch', `전체 (${fmtInt(totalCount)}건 / ${fmtK(totalFee)})`, {kind:'dispatchSecurity', value:''}));
        types.forEach(type => {
            box.appendChild(makeBtn('btn-dispatch', `${type} (${fmtInt(counts[type] || 0)}건 / ${fmtK(fees[type] || 0)})`, {kind:'dispatchSecurity', value:type}));
        });
        const otherKey = '기타/미지정';
        if(counts[otherKey] > 0) {
             box.appendChild(makeBtn('btn-dispatch', `${otherKey} (${fmtInt(counts[otherKey] || 0)}건 / ${fmtK(fees[otherKey] || 0)})`, {kind:'dispatchSecurity', value:otherKey}));
        }

        highlightActive('dispatchSecurity', currentDispatchSecurityFilter);
    }
    function renderQueryTypeButtons(){ const box=document.getElementById('queryBox'); box.innerHTML=''; const TYPES=['정지','설변']; const c={정지:0,설변:0}, f={정지:0,설변:0}; let tC=0,tF=0;
      allData.filter(d=>d.contract && TYPES.includes(d.queryType)).forEach(d=>{ c[d.queryType]++; f[d.queryType]+=d.feeNum||0; }); tC=c.정지+c.설변; tF=(f.정지||0)+(f.설변||0);
      box.appendChild(makeBtn('btn-query', `전체 (${fmtInt(tC)}건 / ${fmtK(tF)})`, {kind:'queryType', value:''}));
      TYPES.forEach(t=> box.appendChild(makeBtn('btn-query', `${t} (${fmtInt(c[t])}건 / ${fmtK(f[t])})`, {kind:'queryType', value:t})));
      highlightActive('queryType', currentQueryType);
    }
    function renderServiceTypeButtons(){ const box=document.getElementById('serviceTypeBox'); box.innerHTML=''; const TYPES=['L형','i형']; const c={'L형':0,'i형':0}, f={'L형':0,'i형':0}; let tC=0,tF=0;
      allData.filter(d=>d.contract).forEach(d=>{ tC++; tF+=d.feeNum||0; const t=(d.serviceType||'').trim(); if(TYPES.includes(t)){ c[t]++; f[t]+=d.feeNum||0; } else { const otherKey = '기타/미지정'; c[otherKey]=(c[otherKey]||0)+1; f[otherKey]=(f[otherKey]||0)+(d.feeNum||0); }});
      box.appendChild(makeBtn('btn-query', `전체 (${fmtInt(tC)}건 / ${fmtK(tF)})`, {kind:'serviceType', value:''}));
      TYPES.forEach(t=> box.appendChild(makeBtn('btn-query', `${t} (${fmtInt(c[t])}건 / ${fmtK(f[t])})`, {kind:'serviceType', value:t})));
      const otherKey = '기타/미지정';
        if(c[otherKey] > 0) {
             box.appendChild(makeBtn('btn-query', `${otherKey} (${fmtInt(c[otherKey] || 0)}건 / ${fmtK(f[otherKey] || 0)})`, {kind:'serviceType', value:otherKey}));
        }
      highlightActive('serviceType', currentServiceType);
    }
    function renderFeeFilterButtons(){ const box=document.getElementById('feeBox'); box.innerHTML=''; let oc=0,uc=0,of=0,uf=0,tC=0,tF=0;
      allData.filter(d=>d.contract).forEach(d=>{ const v=d.feeNum||0; tC++; tF+=v; if(v>=100000){ oc++; of+=v; } else { uc++; uf+=v; }});
      box.appendChild(makeBtn('btn-query', `전체 (${fmtInt(tC)}건 / ${fmtK(tF)})`, {kind:'fee', value:''}));
      box.appendChild(makeBtn('btn-query', `10만원 이상 (${fmtInt(oc)}건 / ${fmtK(of)})`, {kind:'fee', value:'over'}));
      box.appendChild(makeBtn('btn-query', `10만원 이하 (${fmtInt(uc)}건 / ${fmtK(uf)})`, {kind:'fee', value:'under'}));
      highlightActive('fee', feeFilter);
    }
    function renderBranches(){ const box=document.getElementById('branchBox'); box.innerHTML=''; const counts={}, fees={}; let total={정지:0,설변:0};
      branchOrder.forEach(b=> counts[b]={정지:0,설변:0}); // Initialize counts for each branch
      branchOrder.forEach(b=> fees[b]={정지Fee:0,설변Fee:0}); // Initialize fees for each branch
      allData.filter(d=>d.contract && (d.queryType==='정지'||d.queryType==='설변')).forEach(d=>{ const b=d.branch, t=d.queryType; if(branchOrder.includes(b) && (t==='정지'||t==='설변')){ counts[b][t]++; fees[b][t+'Fee']+=d.feeNum||0; total[t]++; }}); // Update counts and fees

      const sumF=(k)=> Object.keys(fees).reduce((s,b)=> s+(fees[b]?.[k+'Fee']||0),0); // Correctly sum fees

      const summary=document.createElement('div'); summary.style.cssText='font-weight:800;background:#0a4f70;padding:.6rem;border-radius:10px;font-size:.93rem;margin-bottom:.7rem;';
      summary.innerHTML=`전체 정지: ${fmtInt(total.정지)}건 (${fmtK(sumF('정지'))}) / 설변: ${fmtInt(total.설변)}건 (${fmtK(sumF('설변'))})`;
      box.appendChild(summary);
      const allCnt = allData.filter(d=>d.contract).length;
      const allFee = allData.filter(d=>d.contract).reduce((sum, d) => sum + (d.feeNum || 0), 0);
      box.appendChild(makeBtn('btn-branch', `전체 (${fmtInt(allCnt)}건 / ${fmtK(allFee)})`, {kind:'branch', value:''}));
      branchOrder.forEach(b=>{ const cj=counts[b]?.정지||0, cs=counts[b]?.설변||0; const fj=fmtK(fees[b]?.정지Fee||0), fs=fmtK(fees[b]?.설변Fee||0); // Use counts and fees for display
        box.appendChild(makeBtn('btn-branch', `${b}\n정지 ${fmtInt(cj)}건 (${fj})\n설변 ${fmtInt(cs)}건 (${fs})`, {kind:'branch', value:b}));
      });
      highlightActive('branch', currentBranch);
    }
    function renderManagers(){
      const box=document.getElementById('managerBox'); box.innerHTML='';
      if(!currentBranch){ box.innerHTML='<p style="color:lightgray;font-size:.9rem;text-align:center;">지사를 선택해주세요.</p>'; return; }

      const agg={};
      let total=0, tf=0;

      // Filter data by currently selected branch for manager display
      const branchFilteredData = allData.filter(d => d.branch === currentBranch && d.contract && (d.queryType === '정지' || d.queryType === '설변'));

      branchFilteredData.forEach(d=>{
        const m=d.manager||'담당자 미지정';
        if(!agg[m]) agg[m]={정지:0,설변:0,정지Fee:0,설변Fee:0};
        if(d.queryType==='정지'){ agg[m].정지++; agg[m].정지Fee += d.feeNum||0; } else { agg[m].설변++; agg[m].설변Fee += d.feeNum||0; }
        total++; tf += d.feeNum||0;
      });

      box.appendChild(makeBtn('btn-manager', `전체 (${fmtInt(total)}건 / ${fmtK(tf)})`, {kind:'manager', value:''}));
      Object.keys(agg).sort().forEach(m=>{
        const a=agg[m];
        box.appendChild(makeBtn('btn-manager', `${m}\n정지 ${fmtInt(a.정지)}건 (${fmtK(a.정지Fee)}) · 설변 ${fmtInt(a.설변)}건 (${fmtK(a.설변Fee)})`, {kind:'manager', value:m}));
      });
      if (currentManager && !agg[currentManager]) currentManager=''; highlightActive('manager', currentManager);

      // After rendering manager buttons, also update manager charts
      renderCharts();
    }

    // ===== 결과 =====
    function applyFilter(){
      console.log('applyFilter: Applying filters - Branch:', currentBranch, 'Manager:', currentManager, 'QueryType:', currentQueryType, 'FeeFilter:', feeFilter, 'ServiceType:', currentServiceType, 'KpiSept:', currentKpiSeptFilter, 'KpiOct:', currentKpiOctFilter, 'DispatchSecurity:', currentDispatchSecurityFilter, 'Channel:', currentChannelFilter); // Added Channel filter log
      filtered=allData.filter(d=>{
        const has=!!d.contract;
        const b=!currentBranch||d.branch===currentBranch;
        const m=!currentManager||d.manager===currentManager;
        const q=!currentQueryType||d.queryType===currentQueryType;
        const f=feeFilter==='' || (feeFilter==='over'&&(d.feeNum||0)>=100000) || (feeFilter==='under'&&(d.feeNum||0)<100000);
        const svc=(d.serviceType||'').trim(); const s=!currentServiceType||svc===currentServiceType || (currentServiceType === '기타/미지정' && !['L형','i형'].includes(svc)); // Handle Other/Undefined for Service Type
        const kpiSept = !currentKpiSeptFilter || (currentKpiSeptFilter === '대상' && d.kpiSept === '대상');
        const kpiOct = !currentKpiOctFilter || (currentKpiOctFilter === '대상' && d.kpiOct === '대상');
        const dispatch = !currentDispatchSecurityFilter || (d.dispatchSecurity || '').trim() === currentDispatchSecurityFilter || (currentDispatchSecurityFilter === '기타/미지정' && !['출동보안','영상보안'].includes((d.dispatchSecurity || '').trim())); // Handle Other/Undefined for Dispatch Security
        const chan = (d.channel || '').trim(); const c=!currentChannelFilter||chan===currentChannelFilter || (currentChannelFilter === '기타/미지정' && !['SP','SC','AM'].includes(chan)); // Handle Channel filter and Other/Undefined

        return has&&b&&m&&q&&f&&s&&kpiSept&&kpiOct&&dispatch&&c; // Include Channel filter in the return condition
      });
      console.log('applyFilter: Filtered data count:', filtered.length);
      console.log('applyFilter: First filtered row example:', filtered.length > 0 ? filtered[0] : 'No filtered data');

      renderResults();
      // Removed renderCharts() from here, it's called in setFilterFromButton and renderManagers
    }

    function renderResults(){
      const box=document.getElementById('resultBox'); box.innerHTML='';
      if (filtered.length===0){ box.innerHTML='<p>🔍 조건에 해당하는 결과가 없습니다.</p>'; return; }
      filtered.forEach(d=>{
        const rid=`${d.rowIndex}-${d.originalRowIndex}`;
        const badge=[];
        if(d.actionType)badge.push(`<b>${String(d.actionType).trim()}</b>`);
        if(d.actionPlan)badge.push(`${String(d.actionPlan).trim()}`);
        if(d.actionMonth)badge.push(`${String(d.actionMonth).trim()}`);
        if(d.kpiSept==='대상') badge.push('<b>KPI 9월말 대상</b>');
        if(d.kpiOct==='대상') badge.push('<b>KPI 10월말 대상</b>');
        if(d.dispatchSecurity) badge.push(`<b>${String(d.dispatchSecurity).trim()}</b>`);
        if(d.channel) badge.push(`<b>${String(d.channel).trim()}</b>`); // Add Channel badge
        // Add Delinquency badge if needed
        if (d.delinquency) badge.push(`<b>${String(d.delinquency).trim()}</b>`);


        const el=document.createElement('div'); el.className='card';
        el.innerHTML=`
          <div class="card-title"><span>${d.name||''} (${d.queryType||''})</span>${badge.length?`<span class="badge-note">참고: ${badge.join(' / ')}</span>`:''}</div>
          <div class="contract">
            <p><strong>계약번호:</strong> ${d.contract||'정보 없음'}</p>
            <p style="background:#fff3cd;padding:.3rem;border-left:5px solid #ffa000;"><strong>월정료:</strong> <strong>${(d.feeNum||0).toLocaleString()}원</strong></p>
            <p><strong>서비스(소):</strong> ${d.serviceType||'정보 없음'}</p>
            <p><strong>정지/설변 시작일:</strong> ${formatDate(d.eventStartDate)||'-'}</p>
            <p><strong>설치주소:</strong> ${(d.address||'').toString().trim()||'정보 없음'}</p>
            ${d.mapUrl?`<p><a href="${d.mapUrl}" target="_blank">📍 지도 보기</a></p>
            `:''}
            <p><strong>지사:</strong> ${d.branch||'정보 없음'}</p>
            <p><strong>구역담당:</strong> ${d.manager||'정보 없음'}</p>
            <p><strong>실적채널:</strong> ${d.channel||'정보 없음'}</p>
            <p><strong>출동보안구분:</strong> ${d.dispatchSecurity||'정보 없음'}</p>
            <p><strong>KPI 차감 9월말:</strong> ${d.kpiSept||'정보 없음'}</p>
            <p><strong>KPI 차감 10월말:</strong> ${d.kpiOct||'정보 없음'}</p>
            <p><strong>체납구분:</strong> ${d.delinquency||'정보 없음'}</p> <!-- Display Delinquency -->
          </div>
          <label for="actionType-${rid}">활동 유형:</label>
          <select id="actionType-${rid}" data-row-index="${d.rowIndex}" data-col-name="actionType" onchange="handleInputChange(this)">
            <option value="">선택</option>
            <option value="유지전환" ${d.actionType==='유지전환'?'selected':''}>유지전환</option>
            <option value="해지정리" ${d.actionType==='해지정리'?'selected':''}>해지정리</option>
            <option value="상담진행" ${d.actionType==='상담진행'?'selected':''}>상담진행</option>
          </select>

          <label for="actionPlan-${rid}">활동 계획:</label>
          <textarea id="actionPlan-${rid}" data-row-index="${d.rowIndex}" data-col-name="actionPlan" rows="3" onchange="handleInputChange(this)">${d.actionPlan||''}</textarea>

          <label for="actionMonth-${rid}">활동 월:</label>
          <select id="actionMonth-${rid}" data-row-index="${d.rowIndex}" data-col-name="actionMonth" onchange="handleInputChange(this)">
            <option value="">선택</option>
            <option value="10월" ${d.actionMonth==='10월'?'selected':''}>10월</option>
            <option value="11월" ${d.actionMonth==='11월'?'selected':''}>11월</option>
            <option value="12월" ${d.actionMonth==='12월'?'selected':''}>12월</option>
          </select>

          <div class="save-row">
            <button class="save-btn" onclick="saveRow(${d.rowIndex})">저장</button>
            <span class="last-save" id="lastSave-${rid}">${d.lastSave?`마지막 저장: ${formatDateTime(d.lastSave)}`:''}</span>
          </div>
        `;
        box.appendChild(el);
      });
    }

    // ===== 저장 =====
    function handleInputChange(el){
      const rowIndex=el.dataset.rowIndex;
      const colName=el.dataset.colName;
      const value=el.value;
      const row=allData.find(d=>d.rowIndex==rowIndex);
      if(row) row[colName]=value;
    }

    function saveRow(rowIndex){
      const row=allData.find(d=>d.rowIndex==rowIndex);
      if(!row){ console.error('Row not found for saving:', rowIndex); showToast('⚠️ 저장 오류: 데이터 불일치'); return; }

      const dataToSave={ rowIndex:row.rowIndex };
      ['actionType','actionPlan','actionMonth'].forEach(col=>{ dataToSave[col]=row[col]||''; });

      console.log('Saving row:', dataToSave);
      document.getElementById('savingOverlay').style.display='flex';

      google.script.run
        .withSuccessHandler(()=>{
          document.getElementById('savingOverlay').style.display='none';
          const rid=`${row.rowIndex}-${row.originalRowIndex}`; // Use originalRowIndex for element ID
          const lastSaveEl=document.getElementById(`lastSave-${rid}`);
          if(lastSaveEl) lastSaveEl.textContent=`마지막 저장: ${formatDateTime(new Date())}`; // Update timestamp
          showToast('✅ 저장 완료');
        })
        .withFailureHandler((e)=>{
          document.getElementById('savingOverlay').style.display='none';
          console.error('Save failed:', e);
          showToast('⚠️ 저장 실패: '+(e?.message||e));
        })
        .saveRowData(dataToSave);
    }

    // ===== 차트 =====
    function setChartType(type){
      currentChartType=type;
      highlightActive('viz', currentChartType);
      renderCharts();
    }

    function renderCharts(){
      // Destroy existing charts
      if(barChartInstance) barChartInstance.destroy();
      if(radarChartInstance) radarChartInstance.destroy();
      if(managerBarChartInstance) managerBarChartInstance.destroy(); // Destroy manager chart
      if(managerRadarChartInstance) managerRadarChartInstance.destroy(); // Destroy manager radar chart

      // Hide manager charts by default
      document.getElementById('managerBarChartContainer').style.display = 'none';
      document.getElementById('managerRadarChartContainer').style.display = 'none';
      document.getElementById('barChartContainer').style.display = 'flex'; // Show branch charts
      document.getElementById('radarChartContainer').style.display = 'flex'; // Show branch radar charts


      let chartData, labels, chartTitle;

      if (currentBranch && !currentManager) {
          // Branch selected, show manager charts
          document.getElementById('barChartContainer').style.display = 'none'; // Hide branch charts
          document.getElementById('radarChartContainer').style.display = 'none'; // Hide branch radar charts
          document.getElementById('managerBarChartContainer').style.display = 'flex'; // Show manager charts
          document.getElementById('managerRadarChartContainer').style.display = 'flex'; // Show manager radar charts


          const managerAgg = {};
          filtered.filter(d => d.branch === currentBranch && d.contract).forEach(d => {
              const m = d.manager || '담당자 미지정';
              if (!managerAgg[m]) managerAgg[m] = { count: 0, fee: 0 };
              managerAgg[m].count++;
              managerAgg[m].fee += d.feeNum || 0;
          });

          labels = Object.keys(managerAgg).sort();
          chartData = labels.map(m => currentChartType === 'count' ? managerAgg[m].count : managerAgg[m].fee);
          chartTitle = `${currentBranch} 지사 구역담당별 ${currentChartType === 'count' ? '건수' : '월정료'} 현황`;

           // Render Manager Bar Chart
          const managerBarCtx = document.getElementById('managerBarChart').getContext('2d');
          managerBarChartInstance = new Chart(managerBarCtx, {
            type:'bar',
            data:{
              labels:labels,
              datasets:[{
                label:chartTitle,
                data:chartData,
                 backgroundColor:labels.map((_,i)=>`hsl(${i*40+180}, 70%, 60%)`), // Different colors for manager charts
                borderColor:labels.map((_,i)=>`hsl(${i*40+180}, 70%, 50%)`),
                borderWidth:1
              }]
            },
            options:{
              responsive:true, maintainAspectRatio:false,
              plugins:{
                title:{ display:true, text:chartTitle, font:{size:16} },
                legend:{ display:false },
                datalabels:{
                  anchor:'end', align:'top',
                  formatter:(value)=> currentChartType==='count'?fmtInt(value)+'건':fmtK(value),
                  color:'#333', font:{ weight:'bold' }
                }
              },
              scales:{ y:{ beginAtZero:true, ticks:{ callback:value=> currentChartType==='count'?fmtInt(value):fmtK(value) } } }
            },
            plugins:[ChartDataLabels]
          });

           // Render Manager Radar Chart
           const managerRadarCtx = document.getElementById('managerRadarChart').getContext('2d');
            managerRadarChartInstance = new Chart(managerRadarCtx, {
                type:'radar',
                data:{
                    labels:labels,
                    datasets:[{
                        label:chartTitle,
                        data:chartData,
                        backgroundColor:'rgba(51, 170, 119, 0.4)', // Different color for manager radar
                        borderColor:'rgba(51, 170, 119, 1)',
                        pointBackgroundColor:'rgba(51, 170, 119, 1)',
                        pointBorderColor:'#fff',
                        pointHoverBackgroundColor:'#fff',
                        pointHoverBorderColor:'rgba(51, 170, 119, 1)'
                    }]
                },
                options:{
                    responsive:true, maintainAspectRatio:false,
                    plugins:{
                        title:{ display:true, text:chartTitle, font:{size:16} },
                        legend:{ display:false },
                        datalabels:{ display:false } // Hide datalabels for radar chart
                    },
                    scales:{ r:{ ticks:{ callback:value=> currentChartType==='count'?fmtInt(value):fmtK(value) } } }
                }
            });


      } else {
          // No branch selected or manager selected, show branch charts
          const branchData={};
          branchOrder.forEach(b=> branchData[b]={count:0,fee:0});
          filtered.filter(d=>d.contract && branchOrder.includes(d.branch)).forEach(d=>{
            branchData[d.branch].count++;
            branchData[d.branch].fee+=d.feeNum||0;
          });

          labels=branchOrder;
          chartData=labels.map(b=> currentChartType==='count'?branchData[b].count:branchData[b].fee);
          chartTitle=currentChartType==='count'?'지사별 건수 현황':'지사별 월정료 현황';

          // Render Branch Bar Chart
          const barCtx=document.getElementById('barChart').getContext('2d');
          barChartInstance=new Chart(barCtx, {
            type:'bar',
            data:{
              labels:labels,
              datasets:[{
                label:chartTitle,
                data:chartData,
                backgroundColor:labels.map((_,i)=>`hsl(${i*40}, 70%, 60%)`),
                borderColor:labels.map((_,i)=>`hsl(${i*40}, 70%, 50%)`),
                borderWidth:1
              }]
            },
            options:{
              responsive:true, maintainAspectRatio:false,
              plugins:{
                title:{ display:true, text:chartTitle, font:{size:16} },
                legend:{ display:false },
                datalabels:{
                  anchor:'end', align:'top',
                  formatter:(value,context)=> currentChartType==='count'?fmtInt(value)+'건':fmtK(value),
                  color:'#333', font:{ weight:'bold' }
                }
              },
              scales:{ y:{ beginAtZero:true, ticks:{ callback:value=> currentChartType==='count'?fmtInt(value):fmtK(value) } } }
            },
            plugins:[ChartDataLabels]
          });

          // Render Branch Radar Chart
          const radarCtx=document.getElementById('radarChart').getContext('2d');
          radarChartInstance=new Chart(radarCtx, {
            type:'radar',
            data:{
              labels:labels,
              datasets:[{
                label:chartTitle,
                data:chartData,
                backgroundColor:'rgba(14, 165, 233, 0.4)',
                borderColor:'rgba(14, 165, 233, 1)',
                pointBackgroundColor:'rgba(14, 165, 233, 1)',
                pointBorderColor:'#fff',
                pointHoverBackgroundColor:'#fff',
                pointHoverBorderColor:'rgba(14, 165, 233, 1)'
              }]
            },
            options:{
              responsive:true, maintainAspectRatio:false,
              plugins:{
                title:{ display:true, text:chartTitle, font:{size:16} },
                legend:{ display:false },
                datalabels:{ display:false } // Hide datalabels for radar chart
              },
              scales:{ r:{ ticks:{ callback:value=> currentChartType==='count'?fmtInt(value):fmtK(value) } } }
            }
          });
      }
    }

    // ===== 유틸리티 =====
    function formatDate(dateStr){
      if(!dateStr) return '';
      try{ const d=new Date(dateStr); return d.toLocaleDateString('ko-KR'); }catch(e){ return dateStr; }
    }
    function formatDateTime(date){
      if(!date) return '';
      try{ const d=new Date(date); return d.toLocaleString('ko-KR'); }catch(e){ return date; }
    }
    function showToast(msg){
      const toast=document.getElementById('toast');
      toast.textContent=msg;
      toast.classList.add('show');
      setTimeout(()=> { toast.classList.remove('show'); }, 3000);
    }

  </script>
</body>
</html>